<h1>Import all libraries and reading explored data into Dataframe</h1>

In [1]:
import re, io, gensim, datetime, time, nltk, random, pickle
from gensim import corpora
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
import pickle
#General libraries needed
import numpy as np
import pandas as pd

#Libraries for data pre-processing
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn import preprocessing

#For Decision Tree implementation
from scipy.stats import entropy
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree

#For Bagging implementation
from sklearn.ensemble import BaggingClassifier
from sklearn.neighbors import KNeighborsClassifier

#For AdaBoost implementation
from sklearn.ensemble import AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB

#For Random Forest implementation
from sklearn.ensemble import RandomForestClassifier

#For Logistic Regression
from sklearn.linear_model import LogisticRegression

#For SVClinear
from sklearn.svm import LinearSVC

string = 'true:'
goldtruth = [string+str(i) for i in range(0, 58)]

string = 'pred:'
prediction = [string+str(i) for i in range(0, 58)]

def printModelAccuracy(y_test, y_pred,file_name):
    # Find the confusion matrix of the result
    cm = pd.DataFrame(confusion_matrix(y_test, y_pred, labels=list(range(0, 58))), \
        index=goldtruth, 
        columns=prediction)
    print("Confusion Matrix:")
    print(cm)
    cm.to_csv(file_name)
    
    # Find the accuracy and F1 score of the result
    asr = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average='micro')
    print("Accuracy:", asr)
    print("F1:", f1)

C:\Users\andy_\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [12]:
X_test = pd.read_pickle("X_test_best_choice_7")
X_train = pd.read_pickle("X_train_best_choice_7")
y_test = pd.read_pickle("y_test_best_choice_7")
y_train = pd.read_pickle("y_train_best_choice_7")

<h1>Naive bayes</h1>

In [7]:
naivebayes = MultinomialNB()
#Fit the training feature Xs and training label Ys
naivebayes.fit(X_train, y_train.values.ravel())

#Use the trained model to predict the test data
y_pred = naivebayes.predict(X_test)

# Find the confusion matrix, the accuracy, and F1 score of the result
#printModelAccuracy(y_test, y_pred,'cm_naive_bayes.csv')


In [35]:
print(X_test.head())
d = {'itemid': X_test.index.tolist(), 'prediction': y_pred}
df = pd.DataFrame(data=d)
df.head()
df.to_csv("naive_bayes_prediction.csv")

           00 000 000mah 001 002 003 006 01 010 0119   ...    zipper zoom  \
itemid                                                 ...                  
95080857    0   0      0   0   0   0   0  0   0    0   ...         0    0   
286502887   0   0      0   0   0   0   0  0   0    0   ...         0    0   
1616581712  0   0      0   0   0   0   0  0   0    0   ...         0    0   
1362929507  0   0      0   0   0   0   0  0   0    0   ...         0    0   
1364551745  0   0      0   0   0   0   0  0   0    0   ...         0    0   

           zoya zr zs620kl zte zuk zv zyrex Category  
itemid                                                
95080857      0  0       0   0   0  0     0       38  
286502887     0  0       0   0   0  0     0       44  
1616581712    0  0       0   0   0  0     0        3  
1362929507    0  0       0   0   0  0     0       34  
1364551745    0  0       0   0   0  0     0        3  

[5 rows x 4908 columns]


<h1>Logistic Regression</h1>

In [36]:
log_reg = LogisticRegression()
#Fit the training feature Xs and training label Ys
log_reg.fit(X_train, y_train.values.ravel())

#Use the trained model to predict the test data
y_pred = log_reg.predict(X_test)

# Find the confusion matrix, the accuracy, and F1 score of the result
#printModelAccuracy(y_test, y_pred,'cm_log_reg.csv')

C:\Users\Admin\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Admin\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


In [37]:
print(X_test.head())
d = {'itemid': X_test.index.tolist(), 'prediction': y_pred}
df = pd.DataFrame(data=d)
df.head()
df.to_csv("logistic_regression_prediction.csv")

           00 000 000mah 001 002 003 006 01 010 0119   ...    zipper zoom  \
itemid                                                 ...                  
95080857    0   0      0   0   0   0   0  0   0    0   ...         0    0   
286502887   0   0      0   0   0   0   0  0   0    0   ...         0    0   
1616581712  0   0      0   0   0   0   0  0   0    0   ...         0    0   
1362929507  0   0      0   0   0   0   0  0   0    0   ...         0    0   
1364551745  0   0      0   0   0   0   0  0   0    0   ...         0    0   

           zoya zr zs620kl zte zuk zv zyrex Category  
itemid                                                
95080857      0  0       0   0   0  0     0       38  
286502887     0  0       0   0   0  0     0       44  
1616581712    0  0       0   0   0  0     0        3  
1362929507    0  0       0   0   0  0     0       34  
1364551745    0  0       0   0   0  0     0        3  

[5 rows x 4908 columns]


<h1>AdaBoost (with Decision Tree)</h1>

In [25]:
#Create the AdaBoost classifier. Default base classifiers is Decision Tree. 
# - n_estimator is the number of base classifiers (i.e. weak learners)
# - learning_rate controls the weight adjustments of each base classifiers. Default is 1
adaboostTree = AdaBoostClassifier(n_estimators=50,learning_rate=1)

#Fit the training feature Xs and training label Ys
adaboostTree.fit(X_train, y_train.values.ravel())

#Use the trained model to predict the test data
y_pred = adaboostTree.predict(X_test)

# Find the confusion matrix, the accuracy, and F1 score of the result
printModelAccuracy(y_test, y_pred,'cm_ada_decision_tree.csv')

Confusion Matrix:
         pred:0  pred:1  pred:2  pred:3  pred:4  pred:5  pred:6  pred:7  \
true:0       58       0       0       0       0       0       0       0   
true:1        0       0       0     426       0       0       0       0   
true:2        0       0       0     156       0       0       0       0   
true:3        0       0       0    1093       0       0       0       0   
true:4        0       0       0       0     619       0       0       0   
true:5        0       0       0       0       0     747       0       0   
true:6        0       0       0       0       0       0       0       0   
true:7        0       0       0       0       0       0       0       0   
true:8        0       0       0       0       0       0       0       0   
true:9        0       0       0       0       0       0       0       0   
true:10       0       0       0       0       0       0       0       0   
true:11       0       0       0       0       0       0       0       0   
true:12

In [4]:
print(X_test.head())
d = {'itemid': X_test.index.tolist(), 'prediction': y_pred}
df = pd.DataFrame(data=d)
df.head()
df.to_csv("adaboost_decision_prediction.csv")

           00 000 000mah 001 002 003 006 01 010 0119   ...    zipper zoom  \
itemid                                                 ...                  
95080857    0   0      0   0   0   0   0  0   0    0   ...         0    0   
286502887   0   0      0   0   0   0   0  0   0    0   ...         0    0   
1616581712  0   0      0   0   0   0   0  0   0    0   ...         0    0   
1362929507  0   0      0   0   0   0   0  0   0    0   ...         0    0   
1364551745  0   0      0   0   0   0   0  0   0    0   ...         0    0   

           zoya zr zs620kl zte zuk zv zyrex Category  
itemid                                                
95080857      0  0       0   0   0  0     0       38  
286502887     0  0       0   0   0  0     0       44  
1616581712    0  0       0   0   0  0     0        3  
1362929507    0  0       0   0   0  0     0       34  
1364551745    0  0       0   0   0  0     0        3  

[5 rows x 4908 columns]


<h1>AdaBoost (with Naive Bayes)</h1>

In [5]:
nb = MultinomialNB()

adaboostnaivebayes = AdaBoostClassifier(n_estimators=50,learning_rate=1, base_estimator=nb)
#model = BaggingClassifier(n_estimators=50, base_estimator=knn)

#Fit the training feature Xs and training label Ys
adaboostnaivebayes.fit(X_train, y_train.values.ravel())

#Use the trained model to predict the test data
y_pred = adaboostnaivebayes.predict(X_test)

# Find the confusion matrix, the accuracy, and F1 score of the result
printModelAccuracy(y_test, y_pred,'cm_ada_boost_naive.csv')

Confusion Matrix:
         pred:0  pred:1  pred:2  pred:3  pred:4  pred:5  pred:6  pred:7  \
true:0        0       0       0       0       0       0       0       0   
true:1        0       0       0       0       0       0       0       0   
true:2        0       0       0       0       0       0       0       0   
true:3        0       0       0       0       0       0       0       0   
true:4        0       0       0       0       0       0       0       0   
true:5        0       0       0       0       0       0       0       0   
true:6        0       0       0       0       0       0       0       0   
true:7        0       0       0       0       0       0       0       0   
true:8        0       0       0       0       0       0       0       0   
true:9        0       0       0       0       0       0       0       0   
true:10       0       0       0       0       0       0       0       0   
true:11       0       0       0       0       0       0       0       0   
true:12

In [6]:
print(X_test.head())
d = {'itemid': X_test.index.tolist(), 'prediction': y_pred}
df = pd.DataFrame(data=d)
df.head()
df.to_csv("adaboost_naive_bayes_prediction.csv")

           00 000 000mah 001 002 003 006 01 010 0119   ...    zipper zoom  \
itemid                                                 ...                  
95080857    0   0      0   0   0   0   0  0   0    0   ...         0    0   
286502887   0   0      0   0   0   0   0  0   0    0   ...         0    0   
1616581712  0   0      0   0   0   0   0  0   0    0   ...         0    0   
1362929507  0   0      0   0   0   0   0  0   0    0   ...         0    0   
1364551745  0   0      0   0   0   0   0  0   0    0   ...         0    0   

           zoya zr zs620kl zte zuk zv zyrex Category  
itemid                                                
95080857      0  0       0   0   0  0     0       38  
286502887     0  0       0   0   0  0     0       44  
1616581712    0  0       0   0   0  0     0        3  
1362929507    0  0       0   0   0  0     0       34  
1364551745    0  0       0   0   0  0     0        3  

[5 rows x 4908 columns]


<h1>Random Forest</h1>

In [3]:
#Instantiate model with 500 decision trees
rf = RandomForestClassifier(n_estimators = 500)

#Fit the training feature Xs and training label Ys
rf.fit(X_train, y_train.values.ravel())

#Use the trained model to predict the test data
y_pred = rf.predict(X_test)

# Find the confusion matrix, the accuracy, and F1 score of the result
printModelAccuracy(y_test, y_pred,'cm_random_forest.csv')

Confusion Matrix:
         pred:0  pred:1  pred:2  pred:3  pred:4  pred:5  pred:6  pred:7  \
true:0       30      10       7       5       3       1       0       0   
true:1        1     395       4      19       6       0       0       0   
true:2        2       8     141       3       1       0       0       0   
true:3        0       4       6    1079       2       1       1       0   
true:4        1       3       0      21     571      12       1       1   
true:5        0       0       1       2       4     738       0       1   
true:6        0       0       0       1       0       0      32       4   
true:7        0       1       0       2       4      13       2     128   
true:8        0       0       0       0       1       3       2       3   
true:9        0       0       0       5       2      19       1       4   
true:10       0       0       0       1       0       0       0       1   
true:11       0       1       1       4       0       2       5       4   
true:12

In [24]:
feature_importances = pd.DataFrame(rf.feature_importances_,
                                   index = X_train.columns,
                                    columns=['Importance']).sort_values('Importance', ascending=False)
feature_importances[1:21]

,Importance
iphone,0.021994
samsung,0.021962
powder,0.018105
oppo,0.017975
dress,0.015390
cream,0.011211
vivo,0.010457
galaxy,0.010278
kaos,0.009495
bb,0.008960


In [8]:
print(X_test.head())
d = {'itemid': X_test.index.tolist(), 'prediction': y_pred}
df = pd.DataFrame(data=d)
df.head()
df.to_csv("random_forest_prediction.csv")

           00 000 000mah 001 002 003 006 01 010 0119   ...    zipper zoom  \
itemid                                                 ...                  
95080857    0   0      0   0   0   0   0  0   0    0   ...         0    0   
286502887   0   0      0   0   0   0   0  0   0    0   ...         0    0   
1616581712  0   0      0   0   0   0   0  0   0    0   ...         0    0   
1362929507  0   0      0   0   0   0   0  0   0    0   ...         0    0   
1364551745  0   0      0   0   0   0   0  0   0    0   ...         0    0   

           zoya zr zs620kl zte zuk zv zyrex Category  
itemid                                                
95080857      0  0       0   0   0  0     0       38  
286502887     0  0       0   0   0  0     0       44  
1616581712    0  0       0   0   0  0     0        3  
1362929507    0  0       0   0   0  0     0       34  
1364551745    0  0       0   0   0  0     0        3  

[5 rows x 4908 columns]


<h1>LinearSVC</h1>

In [9]:
clf = LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=0, tol=1e-05, verbose=0)

clf.fit(X_train, y_train.values.ravel())

#Use the trained model to predict the test data
y_pred = clf.predict(X_test)

# Find the confusion matrix, the accuracy, and F1 score of the result
printModelAccuracy(y_test, y_pred,'cm_linear_svc.csv')


C:\Users\andy_\Anaconda3\lib\site-packages\sklearn\svm\base.py:922: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


Confusion Matrix:
         pred:0  pred:1  pred:2  pred:3  pred:4  pred:5  pred:6  pred:7  \
true:0       35      13       5       2       1       0       0       0   
true:1        1     376       5      25      11       4       1       2   
true:2        0       3     134       5       6       2       1       0   
true:3        0       8       9     967      32      59       1       0   
true:4        0       0       3      51     430      78       2       7   
true:5        0       0       1      34      93     596       0       4   
true:6        0       0       0       2       1       0      19      10   
true:7        0       0       0       2      15       7       5     119   
true:8        0       0       1       1       2       3       4       2   
true:9        0       0       1       4      12      19       0       4   
true:10       0       0       0       0       2       0       0       1   
true:11       0       0       1       0       0       3       4       4   
true:12

In [10]:
print(X_test.head())
d = {'itemid': X_test.index.tolist(), 'prediction': y_pred}
df = pd.DataFrame(data=d)
df.head()
df.to_csv("linearsvc_prediction.csv")

           00 000 000mah 001 002 003 006 01 010 0119   ...    zipper zoom  \
itemid                                                 ...                  
95080857    0   0      0   0   0   0   0  0   0    0   ...         0    0   
286502887   0   0      0   0   0   0   0  0   0    0   ...         0    0   
1616581712  0   0      0   0   0   0   0  0   0    0   ...         0    0   
1362929507  0   0      0   0   0   0   0  0   0    0   ...         0    0   
1364551745  0   0      0   0   0   0   0  0   0    0   ...         0    0   

           zoya zr zs620kl zte zuk zv zyrex Category  
itemid                                                
95080857      0  0       0   0   0  0     0       38  
286502887     0  0       0   0   0  0     0       44  
1616581712    0  0       0   0   0  0     0        3  
1362929507    0  0       0   0   0  0     0       34  
1364551745    0  0       0   0   0  0     0        3  

[5 rows x 4908 columns]


<h1>Bagging</h1>

In [11]:
#Instantiate model with 100 samples & 250 max feature
bagging = BaggingClassifier(base_estimator=None, n_estimators=150, max_samples=70, max_features=300, n_jobs=10)
bagging.fit(X_train, y_train.values.ravel())

#Use the trained model to predict the test data
y_pred = bagging.predict(X_test)

# Find the confusion matrix, the accuracy, and F1 score of the result
printModelAccuracy(y_test, y_pred,'cm_bagging.csv')

Confusion Matrix:
         pred:0  pred:1  pred:2  pred:3  pred:4  pred:5  pred:6  pred:7  \
true:0        0      20       0      36       0       2       0       0   
true:1        0     267       0     133       1      25       0       0   
true:2        0      24       0     130       0       1       0       0   
true:3        0       0       0    1093       0       0       0       0   
true:4        0       0       0      44     568       7       0       0   
true:5        0       0       0       3       0     744       0       0   
true:6        0       0       0       6       0      34       0       0   
true:7        0       0       0      42       0     120       0       0   
true:8        0       0       0      65       0       7       0       0   
true:9        0       0       0      89       3      12       0       0   
true:10       0       0       0      19       0       0       0       0   
true:11       0       0       0      46       0       1       0       0   
true:12

In [12]:
print(X_test.head())
d = {'itemid': X_test.index.tolist(), 'prediction': y_pred}
df = pd.DataFrame(data=d)
df.head()
df.to_csv("bagging_prediction.csv")

           00 000 000mah 001 002 003 006 01 010 0119   ...    zipper zoom  \
itemid                                                 ...                  
95080857    0   0      0   0   0   0   0  0   0    0   ...         0    0   
286502887   0   0      0   0   0   0   0  0   0    0   ...         0    0   
1616581712  0   0      0   0   0   0   0  0   0    0   ...         0    0   
1362929507  0   0      0   0   0   0   0  0   0    0   ...         0    0   
1364551745  0   0      0   0   0   0   0  0   0    0   ...         0    0   

           zoya zr zs620kl zte zuk zv zyrex Category  
itemid                                                
95080857      0  0       0   0   0  0     0       38  
286502887     0  0       0   0   0  0     0       44  
1616581712    0  0       0   0   0  0     0        3  
1362929507    0  0       0   0   0  0     0       34  
1364551745    0  0       0   0   0  0     0        3  

[5 rows x 4908 columns]


<h1>Save the best model and load the best model</h1>

In [ ]:
#save classifier
save_classifier = open("logregmodel.pickle","wb") #binary write
pickle.dump(log_reg, save_classifier)
save_classifier.close()

#Load the save classifier 
classifier_saved = open("logregmodel.pickle", "rb") #binary read
classifier_load = pickle.load(classifier_saved)
classifier_saved.close()
